In [8]:
import numpy as np
import pandas as pd 
import bokeh.io
import bokeh.plotting
import scikit_posthocs as posthoc

bokeh.io.output_notebook()

import urllib.request
# urllib.request.urlretrieve('https://raw.githubusercontent.com/charlesincharge/Caltech-CS155-2022/main/sets/set3/boosting_helper.py', 'boosting_helper.py')

Loading BokehJS ...

In [9]:
df_agm_biod = pd.read_csv("raw-data/extended-data-fig3a.csv")

rng = np.random.default_rng()
df_agm_biod['jitter'] = rng.normal(0, 0.025, len(df_agm_biod)) 
df_agm_biod['cats'] = df_agm_biod.apply(lambda x: (x['tissue'], x['jitter']), axis=1)
df_agm_mean = df_agm_biod.groupby(['tissue', 'variant']).agg(mean_moi=("moi", "mean"), mean_vg=("vg/ug dna", "mean")).reset_index()
df_agm_biod = pd.merge(df_agm_biod, df_agm_mean, on=['tissue','variant'])

df_tissue_biod = df_agm_biod

df_tissue_agg = df_tissue_biod.groupby(['tissue', 'variant'])[['vg/ug dna']].agg(error=('vg/ug dna','std'), n=('vg/ug dna','count')).reset_index()
df_tissue_biod = pd.merge(df_tissue_biod, df_tissue_agg, on=['tissue','variant'], how='right')

df_tissue_biod['error'] = df_tissue_biod['error'] / df_tissue_biod['n'].apply(np.sqrt)
               
df_tissue_biod['upper'] = df_tissue_biod.apply(lambda x: (x['mean_vg'] + x['error'] if x['n'] > 2 else -1), axis=1) 
df_tissue_biod['lower'] = df_tissue_biod.apply(lambda x: (x['mean_vg'] - x['error'] if x['n'] > 2 else -1), axis=1) 

df_tissue_biod.head()

,variant,animal id,tissue,region,moi,vg/ug dna,jitter,cats,mean_moi,mean_vg,error,n,upper,lower
0,AAV9,C002,Adrenal gland,Adrenal gland,30.134134,5.020347e+06,0.024382,"(Adrenal gland, 0.02438249309902714)",19.631298,3.270574e+06,1.749772e+06,2,-1.000000,-1.000000
1,AAV9,C016,Adrenal gland,Adrenal gland,9.128462,1.520802e+06,0.007935,"(Adrenal gland, 0.007934885401792673)",19.631298,3.270574e+06,1.749772e+06,2,-1.000000,-1.000000
2,CAP-Mac,C017,Adrenal gland,Adrenal gland,68.283247,1.137599e+07,0.024512,"(Adrenal gland, 0.02451164294124952)",62.264436,1.037326e+07,1.002734e+06,2,-1.000000,-1.000000
3,CAP-Mac,C010,Adrenal gland,Adrenal gland,56.245625,9.370521e+06,0.003085,"(Adrenal gland, 0.0030853291983145396)",62.264436,1.037326e+07,1.002734e+06,2,-1.000000,-1.000000
4,AAV9,C002,Brain,Caudate,0.674905,1.124392e+05,0.040365,"(Brain, 0.04036469215679239)",0.325003,5.414553e+04,7.421541e+03,22,61567.068723,46723.987099


In [12]:
x_range = ['Brain', 'Spinal cord', 'DRG', 'Kidney', 'Heart', 'Skeletal muscle']

variant_color = bokeh.transform.factor_cmap('variant', palette=["gray", "white"], factors=['AAV9', 'CAP-Mac'])

animal_id = ["C002", "C016", "C010", "C017"]
markers = ['circle', 'triangle', 'square', 'diamond'] 
marker_color=list(bokeh.palettes.Colorblind[6][2:])

############### PLOT SETTINGS ###################
figure_width = 700
figure_height = 350
bar_width = .35
bar_fill_color='gray'
jitter_width = bar_width/2
marker_size = 10
bar_line_width = 1

error_size=5
error_line_width=.25
marker_line_width=1
############### PLOT SETTINGS ###################


p = bokeh.plotting.figure(x_range=bokeh.models.FactorRange(*x_range), title="DNA biodistibution in 8-month old green monkey (intravenous delivery)", 
                          height=figure_height, width=figure_width)

############### PLOT SETTINGS ###################
p.xgrid.visible=False
p.axis.minor_tick_line_width=0
p.xaxis.major_label_orientation=np.pi/4
p.axis.major_tick_in = 0
p.axis.major_label_text_color="#000000"
p.axis.major_label_text_font_size="12pt"
p.xaxis.group_text_font_size="12pt"
p.xaxis.group_text_color="#000000"
p.axis.major_label_standoff=5
p.add_layout(bokeh.models.Legend(), "right")
############### PLOT SETTINGS ###################

p.vbar(source=df_tissue_biod.loc[df_tissue_biod['variant']=='AAV9'], x=bokeh.transform.dodge('tissue', -jitter_width, range=p.x_range), 
       top='mean_vg', bottom=1e-6, width=bar_width, fill_color=variant_color, line_color='black', legend_label="AAV9")
p.vbar(source=df_tissue_biod.loc[df_tissue_biod['variant']=='CAP-Mac'], x=bokeh.transform.dodge('tissue', jitter_width, range=p.x_range), 
       top='mean_vg', bottom=1e-6, width=bar_width, fill_color=variant_color, line_color='black', legend_label="CAP-Mac")

source_error_AAV9 = bokeh.models.ColumnDataSource(data=df_tissue_biod.loc[df_tissue_biod['variant']=='AAV9'])
source_error_C1 = bokeh.models.ColumnDataSource(data=df_tissue_biod.loc[df_tissue_biod['variant']=='CAP-C1'])


w_AAV9 = bokeh.models.Whisker(source=source_error_AAV9, base=bokeh.transform.dodge('tissue', -jitter_width, range=p.x_range), 
                              upper='upper', lower='lower', level='overlay', line_width=error_line_width)
w_C1 = bokeh.models.Whisker(source=source_error_C1, base=bokeh.transform.dodge('tissue', jitter_width, range=p.x_range), 
                            upper='upper', lower='lower', level='overlay', line_width=error_line_width)

w_AAV9.upper_head.line_width=error_line_width
w_AAV9.upper_head.size=error_size
w_AAV9.lower_head.line_width=error_line_width
w_AAV9.lower_head.size=error_size

w_C1.upper_head.line_width=error_line_width
w_C1.upper_head.size=error_size
w_C1.lower_head.line_width=error_line_width
w_C1.lower_head.size=error_size

p.add_layout(w_AAV9)
p.add_layout(w_C1)

y_max = np.ceil(df_tissue_biod.loc[df_tissue_biod["tissue"].isin(x_range)]["vg/ug dna"].max()*1.10)
p.y_range = bokeh.models.Range1d(1, y_max)

p.scatter(x=bokeh.transform.dodge('cats', -jitter_width, range=p.x_range), y='vg/ug dna', 
          source=df_tissue_biod.loc[df_tissue_biod['variant']=='AAV9'], marker=bokeh.transform.factor_mark('animal id', markers, animal_id), size=marker_size,
          color=bokeh.transform.factor_cmap('animal id', marker_color, animal_id), line_color='black', line_width=marker_line_width, legend_group="animal id")

p.scatter(x=bokeh.transform.dodge('cats', jitter_width, range=p.x_range), y='vg/ug dna', 
          source=df_tissue_biod.loc[df_tissue_biod['variant']=='CAP-Mac'], marker=bokeh.transform.factor_mark('animal id', markers, animal_id), size=marker_size,
          color=bokeh.transform.factor_cmap('animal id', marker_color, animal_id), line_color='black', line_width=marker_line_width, legend_group="animal id")

bokeh.io.show(p)

In [13]:
x_range = ['Liver', 'Adrenal gland']
variant_color = bokeh.transform.factor_cmap('variant', palette=["gray", "white"], factors=['AAV9', 'CAP-Mac'])

animal_id = ["C002", "C016", "C010", "C017"]
markers = ['circle', 'triangle', 'square', 'diamond'] 
marker_color=list(bokeh.palettes.Colorblind[6][2:])

############### PLOT SETTINGS ###################
figure_width = 700
figure_height = 350
bar_width = .35
bar_fill_color='gray'
jitter_width = bar_width/2
marker_size = 10
bar_line_width = 1

error_size=5
error_line_width=.25
marker_line_width=1
############### PLOT SETTINGS ###################


p = bokeh.plotting.figure(x_range=bokeh.models.FactorRange(*x_range), title="DNA biodistibution in 8-month old green monkey (intravenous delivery)", 
                          height=figure_height, width=figure_width)

############### PLOT SETTINGS ###################
p.xgrid.visible=False
p.axis.minor_tick_line_width=0
p.xaxis.major_label_orientation=np.pi/4
p.axis.major_tick_in = 0
p.axis.major_label_text_color="#000000"
p.axis.major_label_text_font_size="12pt"
p.xaxis.group_text_font_size="12pt"
p.xaxis.group_text_color="#000000"
p.axis.major_label_standoff=5
p.add_layout(bokeh.models.Legend(), "right")
############### PLOT SETTINGS ###################

p.vbar(source=df_tissue_biod.loc[df_tissue_biod['variant']=='AAV9'], x=bokeh.transform.dodge('tissue', -jitter_width, range=p.x_range), 
       top='mean_vg', bottom=1e-6, width=bar_width, fill_color=variant_color, line_color='black', legend_label="AAV9")
p.vbar(source=df_tissue_biod.loc[df_tissue_biod['variant']=='CAP-Mac'], x=bokeh.transform.dodge('tissue', jitter_width, range=p.x_range), 
       top='mean_vg', bottom=1e-6, width=bar_width, fill_color=variant_color, line_color='black', legend_label="CAP-Mac")

source_error_AAV9 = bokeh.models.ColumnDataSource(data=df_tissue_biod.loc[df_tissue_biod['variant']=='AAV9'])
source_error_C1 = bokeh.models.ColumnDataSource(data=df_tissue_biod.loc[df_tissue_biod['variant']=='CAP-C1'])


w_AAV9 = bokeh.models.Whisker(source=source_error_AAV9, base=bokeh.transform.dodge('tissue', -jitter_width, range=p.x_range), 
                              upper='upper', lower='lower', level='overlay', line_width=error_line_width)
w_C1 = bokeh.models.Whisker(source=source_error_C1, base=bokeh.transform.dodge('tissue', jitter_width, range=p.x_range), 
                            upper='upper', lower='lower', level='overlay', line_width=error_line_width)

w_AAV9.upper_head.line_width=error_line_width
w_AAV9.upper_head.size=error_size
w_AAV9.lower_head.line_width=error_line_width
w_AAV9.lower_head.size=error_size

w_C1.upper_head.line_width=error_line_width
w_C1.upper_head.size=error_size
w_C1.lower_head.line_width=error_line_width
w_C1.lower_head.size=error_size

p.add_layout(w_AAV9)
p.add_layout(w_C1)

y_max = np.ceil(df_tissue_biod.loc[df_tissue_biod["tissue"].isin(x_range)]["vg/ug dna"].max()*1.10)
p.y_range = bokeh.models.Range1d(1, y_max)

p.scatter(x=bokeh.transform.dodge('cats', -jitter_width, range=p.x_range), y='vg/ug dna', 
          source=df_tissue_biod.loc[df_tissue_biod['variant']=='AAV9'], marker=bokeh.transform.factor_mark('animal id', markers, animal_id), size=marker_size,
          color=bokeh.transform.factor_cmap('animal id', marker_color, animal_id), line_color='black', line_width=marker_line_width, legend_group="animal id")

p.scatter(x=bokeh.transform.dodge('cats', jitter_width, range=p.x_range), y='vg/ug dna', 
          source=df_tissue_biod.loc[df_tissue_biod['variant']=='CAP-Mac'], marker=bokeh.transform.factor_mark('animal id', markers, animal_id), size=marker_size,
          color=bokeh.transform.factor_cmap('animal id', marker_color, animal_id), line_color='black', line_width=marker_line_width, legend_group="animal id")

bokeh.io.show(p)

In [14]:
x_range = ['Lung']
variant_color = bokeh.transform.factor_cmap('variant', palette=["gray", "white"], factors=['AAV9', 'CAP-Mac'])

animal_id = ["C002", "C016", "C010", "C017"]
markers = ['circle', 'triangle', 'square', 'diamond'] 
marker_color=list(bokeh.palettes.Colorblind[6][2:])

############### PLOT SETTINGS ###################
figure_width = 700
figure_height = 350
bar_width = .35
bar_fill_color='gray'
jitter_width = bar_width/2
marker_size = 10
bar_line_width = 1

error_size=5
error_line_width=.25
marker_line_width=1
############### PLOT SETTINGS ###################


p = bokeh.plotting.figure(x_range=bokeh.models.FactorRange(*x_range), title="DNA biodistibution in 8-month old green monkey (intravenous delivery)", 
                          height=figure_height, width=figure_width)

############### PLOT SETTINGS ###################
p.xgrid.visible=False
p.axis.minor_tick_line_width=0
p.xaxis.major_label_orientation=np.pi/4
p.axis.major_tick_in = 0
p.axis.major_label_text_color="#000000"
p.axis.major_label_text_font_size="12pt"
p.xaxis.group_text_font_size="12pt"
p.xaxis.group_text_color="#000000"
p.axis.major_label_standoff=5
p.add_layout(bokeh.models.Legend(), "right")
############### PLOT SETTINGS ###################

p.vbar(source=df_tissue_biod.loc[df_tissue_biod['variant']=='AAV9'], x=bokeh.transform.dodge('tissue', -jitter_width, range=p.x_range), 
       top='mean_vg', bottom=1e-6, width=bar_width, fill_color=variant_color, line_color='black', legend_label="AAV9")
p.vbar(source=df_tissue_biod.loc[df_tissue_biod['variant']=='CAP-Mac'], x=bokeh.transform.dodge('tissue', jitter_width, range=p.x_range), 
       top='mean_vg', bottom=1e-6, width=bar_width, fill_color=variant_color, line_color='black', legend_label="CAP-Mac")

source_error_AAV9 = bokeh.models.ColumnDataSource(data=df_tissue_biod.loc[df_tissue_biod['variant']=='AAV9'])
source_error_C1 = bokeh.models.ColumnDataSource(data=df_tissue_biod.loc[df_tissue_biod['variant']=='CAP-C1'])


w_AAV9 = bokeh.models.Whisker(source=source_error_AAV9, base=bokeh.transform.dodge('tissue', -jitter_width, range=p.x_range), 
                              upper='upper', lower='lower', level='overlay', line_width=error_line_width)
w_C1 = bokeh.models.Whisker(source=source_error_C1, base=bokeh.transform.dodge('tissue', jitter_width, range=p.x_range), 
                            upper='upper', lower='lower', level='overlay', line_width=error_line_width)

w_AAV9.upper_head.line_width=error_line_width
w_AAV9.upper_head.size=error_size
w_AAV9.lower_head.line_width=error_line_width
w_AAV9.lower_head.size=error_size

w_C1.upper_head.line_width=error_line_width
w_C1.upper_head.size=error_size
w_C1.lower_head.line_width=error_line_width
w_C1.lower_head.size=error_size

p.add_layout(w_AAV9)
p.add_layout(w_C1)

y_max = np.ceil(df_tissue_biod.loc[df_tissue_biod["tissue"].isin(x_range)]["vg/ug dna"].max()*1.10)
p.y_range = bokeh.models.Range1d(1, y_max)

p.scatter(x=bokeh.transform.dodge('cats', -jitter_width, range=p.x_range), y='vg/ug dna', 
          source=df_tissue_biod.loc[df_tissue_biod['variant']=='AAV9'], marker=bokeh.transform.factor_mark('animal id', markers, animal_id), size=marker_size,
          color=bokeh.transform.factor_cmap('animal id', marker_color, animal_id), line_color='black', line_width=marker_line_width, legend_group="animal id")

p.scatter(x=bokeh.transform.dodge('cats', jitter_width, range=p.x_range), y='vg/ug dna', 
          source=df_tissue_biod.loc[df_tissue_biod['variant']=='CAP-Mac'], marker=bokeh.transform.factor_mark('animal id', markers, animal_id), size=marker_size,
          color=bokeh.transform.factor_cmap('animal id', marker_color, animal_id), line_color='black', line_width=marker_line_width, legend_group="animal id")

bokeh.io.show(p)